In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

### SUB-TASK 1

In [ ]:
pd.DataFrame.fillna(train_features, train_features.mean(), inplace=True)
X = train_features.iloc[:, 3:].to_numpy()
y = train_labels.to_numpy()
clf = LogisticRegression(random_state=0).fit(X, y)

Training della network

In [ ]:
def train_classifier_random_forest(X, y, label):
    y_selected = y[label].to_numpy()
    rf_clf = RandomForestClassifier(n_jobs=4,
        criterion='gini',
        max_depth=32,
        min_samples_split=2,
        min_samples_leaf=16,
        random_state=seed,
        class_weight='balanced',
        n_estimators=200)
    if(cv_random_forest):
        scores = cross_val_score(rf_clf, X, y_selected, scoring='roc_auc', n_jobs=2)
        print(label)
        print("Random forest Roc auc score: %.2f%%" % mean(scores))

    rf_clf.fit(X, y_selected)
    return MedClassifierWrapper(label, rf_clf,
            cross_validation_score=-1,
            clf_params=None)
